In [4]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

In [13]:
MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [14]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [16]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])


In [17]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [19]:
# number of activation layers or hidden layers
nh =50

In [20]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [21]:
def lin(x,w,b): return x@w + b

In [22]:
x_valid.shape

torch.Size([10000, 784])

In [23]:
t = lin(x_valid,w1,b1)
t.shape

torch.Size([10000, 50])

In [24]:
def relu(x): return x.clamp_min(0.)

In [25]:
t =relu(t)
t.shape

torch.Size([10000, 50])

In [26]:
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [27]:
def model_(xb):
  l1 = lin(xb,w1,b1)
  l2 = relu(l1)
  return lin(l2,w2,b2)

In [28]:
res = model_(x_valid)
res.shape

torch.Size([10000, 1])

## LOSS Function
Lets start with MSE to begin with then we return to a better Loss fucntion because MSE is not good with multiclass

In [29]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [33]:
y_valid[:,None].shape

torch.Size([10000, 1])

Here broadcasting is applied. And it starts from Right to Left so first example here res has [10000,1] shape and y_valid has [10000] shape and per the Rules the res value(1) is multiplied with 10000.. right to left rule. But if we change y_valid shape to [10000,1] with y_valid[,:None] it will give us the right result.
### alternatively
### we can change the shape of res to just [10000] by removing the trailing unit vector. With res[:,0] or the sqeeze function res.sqeeze to remove the trailing unit dimentions as the example used below.

In [35]:
(res -y_valid).shape

torch.Size([10000, 10000])

In [34]:
(res-y_valid[:,None]).shape

torch.Size([10000, 1])

In [42]:
res[:,0].shape

torch.Size([10000])

In [39]:
q = res[None,:,None]
q.shape

torch.Size([1, 10000, 1, 1])

In [40]:
q.squeeze().shape

torch.Size([10000])

In [38]:
res.squeeze().shape

torch.Size([10000])

In [43]:
(res[:,0]-y_valid).shape

torch.Size([10000])